In [1]:
import findspark
findspark.init('/home/spark/spark-2.1.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lr_example").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [6]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema = True,header = True)

In [7]:
data.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
data.printSchema()


root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [19]:
data.head(4)[3]

Row(Email='riverarebecca@gmail.com', Address='1414 David ThroughwayPort Jason, OH 22070-1220', Avatar='SaddleBrown', Avg Session Length=34.30555662975554, Time on App=13.717513665142507, Time on Website=36.72128267790313, Length of Membership=3.120178782748092, Yearly Amount Spent=581.8523440352177)

In [21]:
for teamp in data.head(2)[1]:
    print(teamp)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [25]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [27]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [30]:
assembler = VectorAssembler(inputCols= ['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent'],outputCol = 'features')

In [32]:
output = assembler.transform(data)

In [33]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [34]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [38]:
for temp in output.head(1)[0]:
    print(temp)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005
[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615,587.9510539684005]


In [44]:
output.head(1)[0]
output.describe()

DataFrame[summary: string, Email: string, Address: string, Avatar: string, Avg Session Length: string, Time on App: string, Time on Website: string, Length of Membership: string, Yearly Amount Spent: string]

In [45]:
final_data = output.select('features','Yearly Amount Spent')

In [46]:
final_data.head(1)[0]

Row(features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]), Yearly Amount Spent=587.9510539684005)

In [47]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [48]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [49]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                362|
|   mean| 500.04412324376307|
| stddev|  81.83671369234166|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [50]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [51]:
lr_model = lr.fit(train_data)

In [53]:
test_results = lr_model.evaluate(test_data)

In [54]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|2.387423592153936...|
|2.671640686457976...|
|1.648459146963432...|
|-1.70530256582424...|
|-3.63797880709171...|
|-1.93267624126747...|
|-2.84217094304040...|
|-1.25055521493777...|
|5.684341886080801...|
|4.718003765447065...|
|4.433786671143025...|
|-4.54747350886464...|
|-2.55795384873636...|
|-2.72848410531878...|
|-7.38964445190504...|
|1.989519660128280...|
|-1.25055521493777...|
|9.663381206337363...|
|1.136868377216160...|
|1.364242052659392...|
+--------------------+
only showing top 20 rows



In [55]:
test_results.rootMeanSquaredError

2.668475000519491e-12

In [56]:
test_results.r2

1.0

In [57]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [58]:
unlabeled_data = test_data.select('features')

In [59]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.7377203726281...|
|[30.9716756438877...|
|[31.0613251567161...|
|[31.0662181616375...|
|[31.1280900496166...|
|[31.2606468698795...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.6005122003032...|
|[31.6098395733896...|
|[31.8164283341993...|
|[31.8279790554652...|
|[31.8648325480987...|
|[31.9120759292006...|
|[31.9549038566348...|
|[31.9673209478824...|
|[32.0123007682454...|
|[32.0215955013870...|
|[32.0478009788678...|
+--------------------+
only showing top 20 rows

